In [1]:
import python_utils.python_utils.caching as caching
import ml_stuff_run.ml_stuff_run.constants as constants
caching.init(constants.cache_folder, constants.which_hash)
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display_pretty, display_html
import python_utils.python_utils.basic as basic
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as mpl_colors
import matplotlib.cm as cm
import pdb
import numpy as np
import scipy
import ml_stuff.ml_stuff.hier_variational_regr as hvr
import causal_frl.causal_frl.hier_treatment_variational_regr as hvr

/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: mvtnorm

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: Matrix

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: stats4

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: gmm

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: Use

In [3]:
# random seed
np.random.seed(41)

In [4]:
# define params for simulating (x_ns, T_ns)
N = 1500
T_prob = 0.5
K = 4
x_var = 15.0

In [5]:
# simulate (x_ns, T_ns)
x_ns = np.random.multivariate_normal(np.zeros(K), np.eye(K) * x_var, N)
T_ns = np.random.randint(0, 2, N)

In [6]:
# define rule_fs
print hvr
#rule_fs = [hvr.rule(k,0.) for k in xrange(K)]
#rule_fs = [hvr.rule(k,0.) for k in xrange(5)]
all_rule_fs = [hvr.rule(k,0,1) for k in xrange(K)]# + [hvr.rule(k,0,-1) for k in xrange(K)]
#rule_fs = [hvr.mem_address_rule(k,K) for k in xrange(K)]

<module 'causal_frl.causal_frl.hier_treatment_variational_regr' from '/Users/fultonw/Documents/projects/code/causal_frl/causal_frl/hier_treatment_variational_regr.pyc'>


In [7]:
# define p_y_ns_given_theta, theta
#true_rule_f_ls = rule_fs[0:2] + [hvr.default_rule()]
true_rule_f_ls = all_rule_fs[0:3] + [hvr.default_rule()]
#true_rule_f_ls = rule_fs
true_L = len(true_rule_f_ls)
true_p_y_ns_given_theta = hvr.p_y_ns_given_theta(true_rule_f_ls)
true_mu = np.ones(K)*5.0
true_prec = np.eye(K)/1.0
true_lambda_ls = np.ones(true_L) / 1.0
true_delta = np.cumsum(np.arange(1,true_L+1,1))
true_theta = hvr.theta(true_mu, true_prec, np.random.multivariate_normal(true_mu, true_prec, true_L), true_lambda_ls, true_delta)
print 'true delta:', true_delta
print 'true lambdas', true_lambda_ls
print 'true rule_f_ls', true_rule_f_ls
print 'supports', pd.Series(true_p_y_ns_given_theta.get_z_ns_num(x_ns)).value_counts()

true delta: [ 1  3  6 10]
true lambdas [ 1.  1.  1.  1.]
true rule_f_ls [0_0.00_1, 1_0.00_1, 2_0.00_1, default]
supports 0    768
1    386
2    182
3    164
dtype: int64


In [8]:
# simulate y_ns
y_ns = true_p_y_ns_given_theta.sample(true_theta, (x_ns,T_ns))

In [9]:
# define p_theta
v_0 = 60.0001
assert v_0 > K-1
T_0 = 1.0 * np.eye(K) 
c_0 = .0001 
m_0 = np.zeros(K)
alpha_0 = 1.
beta_0 = 1.
r_0 = -1000.
s_0 = 1000.
p_theta = hvr.p_theta(v_0, T_0, c_0, m_0, alpha_0, beta_0, r_0, s_0)

In [10]:
# define get_posterior_f for recovering parameters
recovery_num_iters = 15
recovery_get_posterior_f = hvr.get_posterior_f(recovery_num_iters)

In [11]:
###### for true model, try to recover true theta
true_model = hvr.model(p_theta, true_p_y_ns_given_theta)
v, T, c, m, mu_B_ls, prec_B_ls, alpha_lambda_ls, beta_lambda_ls, mu_delta, prec_delta = recovery_get_posterior_f(true_model, (x_ns, T_ns), y_ns)
print 'v',v
print 'T',T 
print 'c',c
print 'm',m
print 'mu_B_ls', mu_B_ls
print 'prec_B_ls', prec_B_ls
print 'alpha_lambda_ls', alpha_lambda_ls
print 'beta_lambda_ls', beta_lambda_ls
print 'mu_delta', mu_delta
print 'prec_delta',prec_delta
print 'mu_lambda_ls', alpha_lambda_ls / beta_lambda_ls

v 64.0001
T [[ 0.15181365  0.03310026 -0.0788842   0.01058067]
 [ 0.03310026  0.45515285 -0.34628659 -0.06798635]
 [-0.0788842  -0.34628659  0.68424524  0.18137793]
 [ 0.01058067 -0.06798635  0.18137793  0.3666988 ]]
c 4.0001
m [ 5.03878485  4.28519832  4.49730736  5.10748131]
mu_B_ls [[ 6.00230665  5.56313713  5.61441681  4.76271782]
 [ 6.53103839  3.33902874  4.02660111  5.08086463]
 [ 3.86003723  4.20974039  3.87807868  6.30336426]
 [ 3.76226558  4.02933191  4.47057053  4.28349166]]
prec_B_ls [array([[ 11548.28985835,     11.5880366 ,    416.75682338,   -491.3663104 ],
       [    11.5880366 ,  10853.88922085,    -11.89203602,    601.12403566],
       [   416.75682338,    -11.89203602,  10908.23895492,   -200.51302779],
       [  -491.3663104 ,    601.12403566,   -200.51302779,  10929.78196615]]), array([[ 6169.908906  , -3965.08380888,   332.76830896,  -175.51981878],
       [-3965.08380888,  5506.44233894,  -276.9440692 ,   181.88646686],
       [  332.76830896,  -276.9440692 ,  5

In [12]:
# define list of models to calculate evidence for
candidate_models = [hvr.model.model_from_p_theta_and_rule_fs(p_theta, list(rule_fs)) for rule_fs in hvr.subsequence_iterator(all_rule_fs,3)]

In [13]:
# define function to calculate evidence lower bound
num_iters = 15
get_posterior_f = hvr.get_posterior_f(num_iters)
get_evidence_f = hvr.get_evidence_f(get_posterior_f, hvr.log_evidence_lower_bound)

In [14]:
# calculate evidence for each model
results = []

@basic.do_cprofile
def run():
    for candidate_model in candidate_models:
        print candidate_model
        results.append([repr(candidate_model),get_evidence_f(candidate_model, (x_ns, T_ns), y_ns)])
run()

[0_0.00_1, default]
[1_0.00_1, default]
[2_0.00_1, default]
[3_0.00_1, default]
[0_0.00_1, 1_0.00_1, default]
[0_0.00_1, 2_0.00_1, default]
[0_0.00_1, 3_0.00_1, default]
[1_0.00_1, 0_0.00_1, default]
[1_0.00_1, 2_0.00_1, default]
[1_0.00_1, 3_0.00_1, default]
[2_0.00_1, 0_0.00_1, default]
[2_0.00_1, 1_0.00_1, default]
[2_0.00_1, 3_0.00_1, default]
[3_0.00_1, 0_0.00_1, default]
[3_0.00_1, 1_0.00_1, default]
[3_0.00_1, 2_0.00_1, default]
[0_0.00_1, 1_0.00_1, 2_0.00_1, default]
[0_0.00_1, 1_0.00_1, 3_0.00_1, default]
[0_0.00_1, 2_0.00_1, 1_0.00_1, default]
[0_0.00_1, 2_0.00_1, 3_0.00_1, default]
[0_0.00_1, 3_0.00_1, 1_0.00_1, default]
[0_0.00_1, 3_0.00_1, 2_0.00_1, default]
[1_0.00_1, 0_0.00_1, 2_0.00_1, default]
[1_0.00_1, 0_0.00_1, 3_0.00_1, default]
[1_0.00_1, 2_0.00_1, 0_0.00_1, default]
[1_0.00_1, 2_0.00_1, 3_0.00_1, default]
[1_0.00_1, 3_0.00_1, 0_0.00_1, default]
[1_0.00_1, 3_0.00_1, 2_0.00_1, default]
[2_0.00_1, 0_0.00_1, 1_0.00_1, default]
[2_0.00_1, 0_0.00_1, 3_0.00_1, default]


In [15]:
# print evidence for each model
print len(results)
display_html(pd.DataFrame.from_records(results, columns=['rules','log evidence']).sort(columns=['log evidence']).to_html(), raw=True)

40


,rules,log evidence
3,"[3_0.00_1, default]",-9553.725893
12,"[2_0.00_1, 3_0.00_1, default]",-9512.214510
15,"[3_0.00_1, 2_0.00_1, default]",-9508.537845
2,"[2_0.00_1, default]",-9506.757435
33,"[2_0.00_1, 3_0.00_1, 1_0.00_1, default]",-9370.133481
39,"[3_0.00_1, 2_0.00_1, 1_0.00_1, default]",-9363.084728
14,"[3_0.00_1, 1_0.00_1, default]",-9349.139820
32,"[2_0.00_1, 3_0.00_1, 0_0.00_1, default]",-9325.587695
38,"[3_0.00_1, 2_0.00_1, 0_0.00_1, default]",-9320.790582
37,"[3_0.00_1, 1_0.00_1, 2_0.00_1, default]",-9270.996689


In [16]:
# define simulated annealing model fitter
SA_num_iters = 100
temperature = 1.
import pandas as pd
def reject_f(model,(x_ns,T_ns), y_ns): 
    #print 'check', pd.Series(model.p_y_ns_given_theta.get_z_ns_num(x_ns)).value_counts().min() < 10
    ans = np.sum(model.p_y_ns_given_theta.get_z_ns(x_ns), axis=0).min() < 10
    #print ans
    #print np.sum(model.p_y_ns_given_theta.get_z_ns(x_ns), axis=0)
    return ans
    
model_fitter = hvr.simulated_annealing_model_fitter(all_rule_fs, p_theta, SA_num_iters, temperature, get_evidence_f, reject_f)

In [17]:
# run simulated annealing
#best_model, best_evidence = basic.do_cprofile(model_fitter)((x_ns, T_ns), y_ns)
best_model, best_evidence = model_fitter((x_ns, T_ns), y_ns)
print best_model, best_evidence

check initial False
> /Users/fultonw/Documents/projects/code/causal_frl/causal_frl/hier_treatment_variational_regr.py(520)__call__()
-> for i in xrange(self.num_iters):
(Pdb) c
[0_0.00_1, 1_0.00_1, 2_0.00_1, 3_0.00_1] [0_0.00_1, 3_0.00_1]
						step 0
[0_0.00_1, 3_0.00_1, default] -7810.39552893 [0_0.00_1, 3_0.00_1, 1_0.00_1, default] -7678.5324873
[0_0.00_1, 3_0.00_1, 1_0.00_1, default] -7678.5324873 [1_0.00_1, 3_0.00_1, 0_0.00_1, default] -8925.10015052
[0_0.00_1, 1_0.00_1, 2_0.00_1, 3_0.00_1] [0_0.00_1, 3_0.00_1, 1_0.00_1]
[0_0.00_1, 3_0.00_1, 1_0.00_1, default] -7678.5324873 [0_0.00_1, 2_0.00_1, 3_0.00_1, 1_0.00_1, default] -7685.5107595
[0_0.00_1, 1_0.00_1, 2_0.00_1, 3_0.00_1] [0_0.00_1, 3_0.00_1, 1_0.00_1]
[0_0.00_1, 3_0.00_1, 1_0.00_1, default] -7678.5324873 [0_0.00_1, 3_0.00_1, 1_0.00_1, 2_0.00_1, default] -7569.30719534
[0_0.00_1, 3_0.00_1, 1_0.00_1, 2_0.00_1, default] -7569.30719534 [0_0.00_1, 3_0.00_1, 2_0.00_1, default] -7769.52332941
[0_0.00_1, 3_0.00_1, 1_0.00_1, 2_0.00_1